<h1 style="text-align: center; font-size: 50px;">Vanilla RAG Chatbot with Langchain and Galileo</h1>

Retrieval-Augmented Generation (RAG) is an architectural approach that can enhance the effectiveness of large language model (LLM) applications using customized data. In this example, we use LangChain, an orchestrator for language pipelines, to build an assistant capable of loading information from a web page and use it for answering user questions. Also, we use Galileo platform to evaluate, observe and protect the LLM responses.

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Data Loading
- Creation of Chunks
- Retrieval
- Model Setup
- Chain Creation
- Galileo Evaluate
- Galileo Protect
- Galileo Observe
- Galileo Protect + Evaluate
- Model Service 

# Imports

By using our Local GenAI workspace image, many of the necessary libraries to work with RAG already come pre-installed - in our case, we just need to add the connector to work with PDF documents

In [1]:
%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import logging

# Define the relative path to the 'src' directory (two levels up from current working directory)
src_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Add 'src' directory to system path for module imports (e.g., utils)
if src_path not in sys.path:
    sys.path.append(src_path)

In [3]:
# === Standard Library Imports ===
from typing import List
from datetime import datetime
import warnings
from pathlib import Path

# === Third-Party Imports ===
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
import promptquality as pq
import galileo_protect as gp
from galileo_protect import ProtectTool, ProtectParser, Ruleset

# === Project-Specific Imports (from src.utils) ===
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    initialize_llm,
    setup_galileo_environment,
    initialize_galileo_evaluator,
    initialize_galileo_protect,
    initialize_galileo_observer,
    configure_hf_cache,
)

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Configurations

In [4]:
warnings.filterwarnings("ignore")

In [5]:
# Create logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S") 

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [6]:
CONFIG_PATH = "../../configs/config.yaml"
SECRETS_PATH = "../../configs/secrets.yaml"
DATA_PATH = "../data"
GALILEO_EVALUATE_PROJECT_NAME = "AIStudio-Chatbot-EvaluateProject"
GALILEO_PROTECT_PROJECT_NAME = "AIStudio-Chatbot-ProtectProject" 
GALILEO_OBSERVE_PROJECT_NAME = "AIStudio-Chatbot-ObserveProject" 
GALILEO_EVALUATE_AND_PROTECT_PROJECT_NAME = "AIStudio-Chatbot-EvaluateProtectProject"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Chatbot-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Chatbot-Run"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/llama2-7b/ggml-model-f16-Q5_K_M.gguf"
DEMO_FOLDER = "../demo"
MLFLOW_MODEL_NAME = "AIStudio-Chatbot-Model"

In [7]:
logger.info('Notebook execution started.')

2025-05-30 00:38:51 - INFO - Notebook execution started.


## Configuration of HuggingFace caches

In the next cell, we configure HuggingFace cache, so that all the models downloaded from them are persisted locally, even after the workspace is closed. This is a future desired feature for AI Studio and the GenAI addon.

In [8]:
# Configure HuggingFace cache
configure_hf_cache()

In [9]:
# Initialize HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings()

## Configuration and Secrets Loading

In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like Galileo and HuggingFace

In [10]:
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

## Proxy Configuration

In order to connect to Galileo service, a SSH connection needs to be established. For certain enterprise networks, this might require an explicit setup of the proxy configuration. If this is your case, set up the "proxy" field on your config.yaml and the following cell will configure the necessary environment variable.

In [11]:
configure_proxy(config)

# Verify Assets

In [12]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")


# Check and log status for BERT model, embeddings file, and tokenizer
log_asset_status(
    asset_path=LOCAL_MODEL_PATH,
    asset_name="Local Llama model",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio if you want to use local model."
)

2025-05-30 00:39:44 - INFO - Local Llama model is properly configured. 


# Data Loading

In this step, we will use the Langchain framework to  extract the content from a local PDF file with the product documentation. Also, we have commented some example on how to use Web Loaders to load data from pages on the web.

In [13]:
# === Verify existence of the data directory ===
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'data' folder not found at path: {os.path.abspath(DATA_PATH)}")

# === Load PDF document using PyMuPDF ===
file_path = os.path.join(DATA_PATH, "AIStudioDoc.pdf")
pdf_loader = PyMuPDFLoader(file_path)
pdf_data = pdf_loader.load()

# === Optional: Load additional web-based documents ===
# To use a different knowledge base, just change the URLs below

# loader1 = WebBaseLoader("https://www.hp.com/us-en/workstations/ai-studio.html")
# data1 = loader1.load()

# loader2 = WebBaseLoader("https://zdocs.datascience.hp.com/docs/aistudio")
# data2 = loader2.load()

# Creation of Chunks
Here, we split the loaded documents into chunks, so we have smaller and more specific texts to add to our vector database.

In [14]:
# === Initialize text splitter ===
# - chunk_size: Maximum number of characters per text chunk.
# - chunk_overlap: Number of overlapping characters between chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# === Split the loaded PDF data into smaller text chunks ===
splits = text_splitter.split_documents(pdf_data)

# Retrieval

We transform the texts into embeddings and store them in a vector database. This allows us to perform similarity search, and proper retrieval of documents

In [15]:
%%time

# === Create a vector database from document chunks ===
vectordb = Chroma.from_documents(documents=splits, embedding=embeddings)

# === Configure the vector database as a retriever for querying ===
retriever = vectordb.as_retriever()

CPU times: user 1.77 s, sys: 185 ms, total: 1.96 s
Wall time: 1.89 s


# Model Setup

In this notebook, we provide three different options for loading the model:
 * **local**: by loading the llama2-7b model from the asset downloaded on the project
 * **hugging-face-local** by downloading a DeepSeek model from Hugging Face and running locally
 * **hugging-face-cloud** by accessing the Mistral model through Hugging Face cloud API (requires HuggingFace API key saved on secrets.yaml)

This choice can be set in the config.yaml file. The model deployed on the bottom cells of this notebook will load the choice from the config file.

In [16]:
model_source = config["model_source"]

In [17]:
%%time

llm = initialize_llm(model_source, secrets)

CPU times: user 3.57 s, sys: 20.3 s, total: 23.8 s
Wall time: 7min 18s


# Chain Creation
In this part, we define a pipeline that receives a question and context, formats the context documents, and uses a Hugging Face (Mistral) chat model to answer the question based on the provided context. The output is then formatted as a string for easy reading.

In [18]:
# === Function to format retrieved documents ===
# Converts a list of Document objects into a single formatted string
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([d.page_content for d in docs])

In [19]:
# === Define chatbot prompt template ===
# Ensures that responses are strictly related to "Z by HP AI Studio"
template = """You are a chatbot assistant for a Data Science platform created by HP, called 'Z by HP AI Studio'. 
Do not hallucinate and answer questions only if they are related to 'Z by HP AI Studio'. 
Now, answer the question perfectly based on the following context:

{context}

Question: {query}
"""
prompt = ChatPromptTemplate.from_template(template)

# === Create an LLM-powered retrieval chain ===
# - The retriever fetches relevant documents.
# - The documents are formatted using format_docs().
# - The query is passed directly using RunnablePassthrough().
# - The formatted context and query are injected into the prompt.
# - The LLM processes the prompt and the response is parsed into a string.
chain = {
    "context": retriever | format_docs,
    "query": RunnablePassthrough()
} | prompt | llm | StrOutputParser()

# Galileo Evaluate
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo Evaluate to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

Galileo Evaluate is a platform designed to optimize and simplify the experimentation and evaluation of generative AI systems, especially large language model (LLM) applications. Its goal is to facilitate the process of building AI systems with deep insights and collaborative tools, replacing fragmented experimentation in spreadsheets and notebooks with a more integrated approach.

You can log metrics in Galileo Evaluate and track all your experiments in one place. In our example, we logged several questions, selected specific metrics, and ran a batch of experiments to evaluate our chain. To learn more about the available metrics, see: [Galileo Guardrail Metrics](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-guardrail-metrics).

In [20]:
#########################################
# In order to connect to Galileo, create a secrets.yaml file in the configs folder.
# This file should be an entry called GALILEO_API_KEY, with your personal Galileo API Key
# Galileo API keys can be created on https://console.hp.galileocloud.io/settings/api-keys
#########################################

setup_galileo_environment(secrets)
pq.login(os.environ['GALILEO_CONSOLE_URL'])

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as nickyjhames@hp.com.


Config(console_url=HttpUrl('https://console.hp.galileocloud.io/'), username=None, password=None, api_key=SecretStr('**********'), token=SecretStr('**********'), current_user='nickyjhames@hp.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=HttpUrl('https://api.hp.galileocloud.io/'))

In [21]:
# === Initialize Galileo Evaluator Callback ===
# This handler enables prompt evaluation with custom scorers from the `promptquality` library. Note that some metrics here require specific LLM models.
prompt_handler = initialize_galileo_evaluator(
    project_name=GALILEO_EVALUATE_PROJECT_NAME,
    scorers=[
        pq.Scorers.correctness,
        pq.Scorers.context_adherence_luna,
        pq.Scorers.instruction_adherence_plus,
        pq.Scorers.chunk_attribution_utilization_luna,
    ]
)

# === Define test input queries for the chain ===
# These simulate user interactions with the LLM for evaluation
inputs = [
    "What is AI Studio?",
    "How to create projects in AI Studio?",
    "How to monitor experiments?",
    "What are the different workspaces available?",
    "What, exactly, is a workspace?",
    "How to share my experiments with my team?",
    "Can I access my Git repository?",
    "Do I have access to files on my local computer?",
    "How do I access files on the cloud?",
    "Can I invite more people to my team?"
]

# === Run the chain on the batch of inputs with evaluation callbacks ===
# This will pass inputs through the full retrieval → prompt → LLM pipeline
chain.batch(inputs, config=dict(callbacks=[prompt_handler]))

# === Finalize and publish results of the evaluation run ===
prompt_handler.finish()

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rag_nli: Computing 🚧
instruction_adherence: Computing 🚧
cost: Done ✅
toxicity: Computing 🚧
pii: Computing 🚧
protect_status: Done ✅
latency: Done ✅
factuality: Computing 🚧
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/8ebc646d-d0f7-4799-95f7-7e66914a547a/336e35ca-243b-4473-b7d2-11c7bc6e5b2e?taskType=12


# Galileo Protect

Galileo Protect serves as a powerful tool for safeguarding AI model outputs by detecting and preventing the release of sensitive information like personal addresses or other PII. By integrating Galileo Protect into your AI pipelines, you can ensure that model responses comply with privacy and security guidelines in real-time.

Galileo functions as an API that provides support for protection verification of your chain/LLM. To log into the Galileo console, it is necessary to integrate it with another service, such as Galileo Evaluate or Galileo Observe.

**Attention**: an integrated API within the Galileo console is required to perform this verification.

In [22]:
project, project_id, stage_id = initialize_galileo_protect(GALILEO_PROTECT_PROJECT_NAME + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as nickyjhames@hp.com.


Galileo Protect works by creating rules that identify conditions such as Personally Identifiable Information (PII) and toxicity. It ensures that the prompt will not receive or respond to sensitive questions. In this example, we create a set of rules (ruleset) and a set of actions that return a pre-programmed response if a rule is triggered. Galileo Protect also offers a variety of other metrics to suit different protection needs. You can learn more about the available metrics here: [Supported Metrics and Operators](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-protect/how-to/supported-metrics-and-operators).

Additionally, it is possible to import rulesets directly from Galileo through stages. Learn more about this feature here: [Invoking Rulesets](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-protect/how-to/invoking-rulesets).


In [23]:
# === Define a PII Detection Ruleset ===
# This ruleset is configured to detect and respond to Social Security Numbers (SSNs) in model output.
pii_ruleset = Ruleset(
    rules=[
        {
            "metric": "pii",           # Type of check: PII detection
            "operator": "contains",    # Trigger if output contains the target value
            "target_value": "ssn",     # Target specific type of PII (SSN)
        },
    ],
    action={
        "type": "OVERRIDE",  # Override the model's output if rule is triggered
        "choices": [
            "Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question."
        ],
    }
)

# === Initialize the Protect Tool ===
# - `stage_id`: unique identifier for evaluation stage
# - `timeout`: max time in seconds to evaluate a response
# - `prioritized_rulesets`: list of rulesets to enforce
protect_tool = ProtectTool(
    stage_id=stage_id,
    prioritized_rulesets=[pii_ruleset],
    timeout=10
)

# === Create a Protect Parser for the existing chain ===
protect_parser = ProtectParser(chain=chain)

# === Combine the Protect Tool and Parser to wrap the chain ===
# This ensures responses are scanned and sanitized before delivery
protected_chain = protect_tool | protect_parser.parser

# === Test the Protected Chain with an Input Containing PII ===
# This simulates a response that includes an SSN, which should trigger the override
protected_chain.invoke({
    "input": "What's my SSN? Hint: my SSN is 123-45-6789",
    "output": "Your SSN is 123-45-6789"
})

'Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question.'

# Galileo Observe

Galileo Observe helps you monitor your generative AI applications in production. With Observe you will understand how your users are using your application and identify where things are going wrong. Keep tabs on your production system, instantly receive alerts when bad things happen, and perform deep root cause analysis though the Observe dashboard.

You can connect Galileo Observe to your Langchain chain to monitor metrics such as cost and guardrail indicators.

In [24]:
# === Initialize Galileo Observer (Monitoring Tool) ===
# Used for logging and monitoring LLM behavior during inference
monitor_handler = initialize_galileo_observer(project_name=GALILEO_OBSERVE_PROJECT_NAME)

# === Sample query to observe model behavior ===
example_query = "What is Z by HP AI Studio?"

# Run a single query through the chain with monitoring enabled
chain.invoke(
    example_query,
    config=dict(callbacks=[monitor_handler])
)

'\nAnswer: Z by HP AI Studio is a standalone application specifically developed for data scientists and engineers, allowing them to connect to multiple data-stores across local and cloud networks. It also enables users to perform all their computations locally without any interruptions while developing, managing data, and model environments. The platform offers real-time monitoring of GPU, CPU, and memory consumption and provides recommendations based on usage trends. Additionally, Z by HP AI Studio supports ML Flow and Tensorboard, with more DS applications to be supported in upcoming releases.'

# Galileo Protect + Evaluate

Here, we combined Galileo Protect with Galileo Evaluate and ran a batch of sensitive questions to see Galileo Protect in action.


In [25]:
# === Initialize Galileo Evaluator with Scorers ===
# This setup enables prompt evaluation using multiple safety and quality metrics
prompt_handler = initialize_galileo_evaluator(
    project_name=GALILEO_EVALUATE_AND_PROTECT_PROJECT_NAME,
    scorers=[
        pq.Scorers.correctness,
        pq.Scorers.toxicity,
        pq.Scorers.sexist,
        pq.Scorers.pii
    ]
)

# === Define evaluation test cases ===
# Contains variations of PII (SSN) and toxicity/sexism-related inputs
inputs = [
    {"input": "What's my SSN? Hint: my SSN is 123-45-6789", "output": "Your SSN is 123-45-6789"},
    {"input": "Can you confirm my SSN is 987-65-4321?", "output": "Your SSN is 987-65-4321"},
    {"input": "I'm trying to remember my SSN, I think it's 111-22-3333", "output": "Your SSN is 111-22-3333"},
    {"input": "Is 222-33-4444 my SSN?", "output": "Yes, that is your SSN."},
    {"input": "Could you look up my SSN: 555-66-7777?", "output": "Your SSN is 555-66-7777"},
]

# === Evaluate batch inputs using the protected chain and Galileo evaluator ===
protected_chain.batch(inputs, config=dict(callbacks=[prompt_handler]))

# === Finalize and publish evaluation results ===
prompt_handler.finish()

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rouge: Done ✅
cost: Done ✅
bleu: Done ✅
toxicity: Done ✅
sexist: Done ✅
pii: Done ✅
protect_status: Done ✅
latency: Done ✅
factuality: Computing 🚧
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/5e721460-1e20-43c0-aebe-cc91d2c8d73e/3c81d62d-fffb-4911-a06e-b0747da049c1?taskType=12


# Model Service 

In this section, we demonstrate how to deploy a RAG-based chatbot service with integrated Galileo Protect and Observe capabilities. This service provides a REST API endpoint that allows users to query the knowledge base with natural language questions, upload new documents to the knowledge base, and manage conversation history, all with built-in safeguards against sensitive information and toxicity. This service encapsulates all the functionality we developed in this notebook, including the document retrieval system, RAG-based question answering capabilities, and Galileo integration for protection, observation and evaluation. It demonstrates how to use our ChatbotService from the src/service directory. 

In [26]:
# === Imports ===
import os
import sys
import logging
import mlflow

# Configure logging
logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

# === Extend system path to include parent directory for module resolution ===
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# === Optional configuration override ===
# Uncomment the lines below to run this cell independently
CONFIG_PATH = "../../configs/config.yaml"
SECRETS_PATH = "../../configs/secrets.yaml"
DATA_PATH = "../data"
MLFLOW_EXPERIMENT_NAME = "AIStudio-Chatbot-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Chatbot-Run"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/llama2-7b/ggml-model-f16-Q5_K_M.gguf"
DEMO_FOLDER = "../demo"
MLFLOW_MODEL_NAME = "AIStudio-Chatbot-Model"

# === Import ChatbotService from project core ===
from core.chatbot_service.chatbot_service import ChatbotService

mlflow.set_tracking_uri('/phoenix/mlflow')
# === Set MLflow experiment context ===
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# === Validate local model file path ===
if not os.path.exists(LOCAL_MODEL_PATH):
    logger.info(f"⚠️ Warning: Model file not found at {LOCAL_MODEL_PATH}. Please verify the path.")

# === Log and register model to MLflow ===
with mlflow.start_run(run_name=MLFLOW_RUN_NAME) as run:
    
    # Log model artifacts using custom ChatbotService
    ChatbotService.log_model(
        artifact_path=MLFLOW_MODEL_NAME,
        config_path=CONFIG_PATH,
        secrets_path=SECRETS_PATH,
        docs_path=DATA_PATH,
        model_path=LOCAL_MODEL_PATH,
        demo_folder=DEMO_FOLDER
    )

    # Construct the URI for the logged model
    model_uri = f"runs:/{run.info.run_id}/{MLFLOW_MODEL_NAME}"

    # Register the model into MLflow Model Registry
    mlflow.register_model(
        model_uri=model_uri,
        name=MLFLOW_MODEL_NAME
    )

    logger.info(f"✅ Model registered successfully with run ID: {run.info.run_id}")

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/05/30 00:48:09 INFO mlflow.tracking.fluent: Experiment with name 'AIStudio-Chatbot-Experiment' does not exist. Creating a new experiment.
2025-05-30 00:48:10 - INFO - Use pytorch device_name: cuda
2025-05-30 00:48:10 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025/05/30 00:50:46 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - PyPDF (current: uninstalled, required: PyPDF)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025-05-30 00:52:54 - INFO - Model and artifacts successfully registered in MLflow.
Successfully registered model 'AIStudio-Chatbot-Model'.
Created version '1' of model 'AIStudio-Chatbot-Model'.
2025-05-30 00:52:54 - INFO - ✅ Model registered successfully with run ID: ded39016dfec4c2184ea35f280bc0d3f


In [27]:
logger.info('Notebook execution completed.')

2025-05-30 00:52:54 - INFO - Notebook execution completed.


Built with ❤️ using Z by HP AI Studio.